### data 

# 📊 Refactoring NMR Analysis for Reusability

This notebook demonstrates how to **modularize and refactor** the original NMR analysis workflow into clean, reusable components. The aim is to create a flexible and maintainable codebase for future applications, including:

- 📁 **Data loading**
- 🔄 **Fourier transform**
- 🔍 **Peak detection**
- 📈 **Integration**
- 🧪 **Functional group identification**
- ⚛️ **Quantum-level simulation**

All the functions and classes used here are refactored from the original `main_project.ipynb` file.

By restructuring the code into **functions and classes**, we enable:
- 🚀 Faster prototyping
- 🔧 Easier debugging and testing
- ♻️ Reusability across projects
- 📦 Potential packaging into libraries or APIs


## 1. Encapsulate Data Loading and Preprocessing

We'll create a function `load_nmr_data(filepath)` to load and preprocess the NMR data. This function can be extended to handle different file formats or preprocessing steps.

In [1]:
import numpy as np
import pandas as pd

def load_nmr_data(filepath, delimiter='\t', skip_header=1, columns=('Time', 'Real', 'Imaginary')):
    """
    Load NMR data from a file and return as a DataFrame.

    Parameters:
        filepath (str): Path to the data file.
        delimiter (str): Delimiter used in the file.
        skip_header (int): Number of header lines to skip.
        columns (tuple): Column names for the DataFrame.

    Returns:
        pd.DataFrame: DataFrame with columns ['Time', 'Real', 'Imaginary'].
    """
    data = np.genfromtxt(filepath, delimiter=delimiter, skip_header=skip_header)
    df = pd.DataFrame(data, columns=columns)
    return df

In [3]:
file_path = "13_03_11_indst_1H fid.asc"
df = load_nmr_data(file_path)
print(df.head())

       Time      Real  Imaginary
0  0.000000  0.000003   0.000009
1  0.000133 -0.001235  -0.003140
2  0.000267  0.050601   0.188029
3  0.000400  0.019175   0.365893
4  0.000533 -0.136525  -0.611649


## 2. Refactor Nyquist Frequency Calculation into a Function

We'll define a function `calculate_nyquist_frequency(time_array)` that takes an array of time points and returns the Nyquist frequency. This makes the calculation reusable and clear.

In [4]:
def calculate_nyquist_frequency(time_array):
    """
    Calculate the Nyquist frequency from an array of time points.

    Parameters:
        time_array (np.ndarray): Array of time points (in seconds).

    Returns:
        float: Nyquist frequency in Hz.
    """
    if len(time_array) < 2:
        raise ValueError("Need at least two time points to calculate dwell time.")
    dwell_time = time_array[1] - time_array[0]
    nyquist_freq = 1 / (2 * dwell_time)
    return nyquist_freq

In [ ]:
nyquist_freq = calculate_nyquist_frequency(df["Time"].values)
print(f"Nyquist frequency: {nyquist_freq:.2f} Hz")


Nyquist frequency: 3751.50 Hz


## 3. Create a Class for NMR Spectrum Analysis

We'll define a class `NMRSpectrum` that encapsulates the workflow: loading data, performing FFT, detecting peaks, integrating, and plotting. This class will store relevant attributes and provide methods for each analysis step.

In [6]:
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks
from scipy.integrate import simpson

class NMRSpectrum:
    def __init__(self, time, real, imaginary=None, name=None, spectrometer_freq=None):
        self.time = np.array(time)
        self.real = np.array(real)
        self.imaginary = np.array(imaginary) if imaginary is not None else None
        self.name = name
        self.spectrometer_freq = spectrometer_freq
        self.fft_result = None
        self.frequencies = None
        self.fft_magnitude = None
        self.peaks = None
        self.peak_properties = None
        self.integrated_areas = None

    @classmethod
    def from_dataframe(cls, df, name=None, spectrometer_freq=None):
        return cls(df['Time'], df['Real'], df.get('Imaginary', None), name, spectrometer_freq)

    def compute_fft(self):
        dwell_time = self.time[1] - self.time[0]
        self.fft_result = fft(self.real)
        self.frequencies = fftfreq(len(self.real), d=dwell_time)
        self.fft_magnitude = np.abs(self.fft_result)
        return self.frequencies, self.fft_magnitude

    def detect_peaks(self, height_ratio=0.1, min_distance=100, min_prominence=0.05):
        if self.fft_magnitude is None:
            raise ValueError("Run compute_fft() first.")
        height_threshold = height_ratio * np.max(self.fft_magnitude)
        min_prom = min_prominence * np.max(self.fft_magnitude)
        peaks, properties = find_peaks(
            self.fft_magnitude,
            height=height_threshold,
            distance=min_distance,
            prominence=min_prom
        )
        self.peaks = peaks
        self.peak_properties = properties
        return peaks, properties

    def integrate_peaks(self):
        if self.peaks is None or self.peak_properties is None:
            raise ValueError("Run detect_peaks() first.")
        integrated_areas = []
        for i, peak_idx in enumerate(self.peaks):
            left = self.peak_properties['left_bases'][i]
            right = self.peak_properties['right_bases'][i]
            freq_region = self.frequencies[left:right+1]
            mag_region = self.fft_magnitude[left:right+1]
            area = simpson(mag_region, freq_region)
            integrated_areas.append(area)
        self.integrated_areas = integrated_areas
        return integrated_areas

    def normalize_integrals(self):
        if self.integrated_areas is None:
            raise ValueError("Run integrate_peaks() first.")
        min_area = min(self.integrated_areas)
        return [area / min_area for area in self.integrated_areas]

## 4. Modularize Fourier Transform and Frequency Axis Calculation

The Fourier transform logic is encapsulated in the `compute_fft` method of the `NMRSpectrum` class. You can also define a standalone function if needed.

In [ ]:
def compute_fft(real_signal, dwell_time):
    """
    Compute the FFT, frequency axis, and magnitude for a real signal.

    Parameters:
        real_signal (np.ndarray): The real part of the signal.
        dwell_time (float): Time between points (seconds).

    Returns:
        tuple: (frequencies, fft_result, fft_magnitude)
    """
    fft_result = fft(real_signal)
    frequencies = fftfreq(len(real_signal), d=dwell_time)
    fft_magnitude = np.abs(fft_result)
    return frequencies, fft_result, fft_magnitude

## 5. Encapsulate Peak Detection and Integration

Peak detection and integration are provided as methods in the `NMRSpectrum` class, but you can also use standalone functions for flexibility.

In [ ]:
def detect_peaks(magnitude, height_ratio=0.1, min_distance=100, min_prominence=0.05):
    """
    Detect peaks in a magnitude array.

    Returns:
        tuple: (peaks, properties)
    """
    height_threshold = height_ratio * np.max(magnitude)
    min_prom = min_prominence * np.max(magnitude)
    peaks, properties = find_peaks(
        magnitude,
        height=height_threshold,
        distance=min_distance,
        prominence=min_prom
    )
    return peaks, properties

def integrate_peaks(frequencies, magnitudes, peaks, properties):
    """
    Integrate each detected peak using Simpson's rule.

    Returns:
        list: Integrated areas for each peak.
    """
    areas = []
    for i, peak_idx in enumerate(peaks):
        left = properties['left_bases'][i]
        right = properties['right_bases'][i]
        freq_region = frequencies[left:right+1]
        mag_region = magnitudes[left:right+1]
        area = simpson(mag_region, freq_region)
        areas.append(area)
    return areas

## 6. Create Functional Group Identification as a Function

Refactor the functional group identification logic into a reusable function.

In [ ]:
def identify_functional_groups(frequencies, magnitudes, ppm_shifts):
    """
    Identify functional groups based on peak positions in the spectrum.

    Parameters:
        frequencies (np.ndarray): Frequencies in ppm.
        magnitudes (np.ndarray): Magnitudes of the FFT result.
        ppm_shifts (dict): Mapping of functional groups to their ppm ranges.

    Returns:
        list: List of (peak_ppm, group) tuples.
    """
    peaks, _ = find_peaks(magnitudes, height=0.1 * max(magnitudes))
    peak_positions = frequencies[peaks]
    identified_groups = []
    for peak in peak_positions:
        for group, ppm_range in ppm_shifts.items():
            ppm_min, ppm_max = map(float, ppm_range.replace("ppm", "").split("-"))
            if ppm_min <= peak <= ppm_max:
                identified_groups.append((peak, group))
                break
    return identified_groups

## 7. Encapsulate Multiplet and J-Coupling Analysis

Write functions for multiplet detection and J-coupling estimation.

In [ ]:
def analyze_multiplet(ppm_axis, intensity, center_ppm, spectrometer_freq, window=0.05):
    """
    Analyze a multiplet region and estimate J-couplings.

    Returns:
        tuple: (sub_ppms, j_couplings)
    """
    mask = (ppm_axis > center_ppm - window) & (ppm_axis < center_ppm + window)
    sub_peaks, _ = find_peaks(intensity[mask], height=0.1 * max(intensity[mask]))
    sub_ppms = ppm_axis[mask][sub_peaks]
    sub_ppms = np.sort(sub_ppms)
    j_couplings = []
    for i in range(1, len(sub_ppms)):
        ppm_diff = abs(sub_ppms[i] - sub_ppms[i - 1])
        j_hz = ppm_diff * spectrometer_freq
        j_couplings.append(j_hz)
    return sub_ppms, j_couplings

## 8. Modularize Quantum Simulation Components

Encapsulate quantum simulation logic into a class, e.g., `SpinHamiltonian`, with methods for building the Hamiltonian, solving eigenstates, and animating time evolution.

In [ ]:
import numpy as np

class SpinHamiltonian:
    def __init__(self, n_spins, j_couplings=None, bx=5):
        self.n_spins = n_spins
        self.j_couplings = j_couplings if j_couplings is not None else [7] * (n_spins - 1)
        self.bx = bx
        self.I = np.eye(2)
        self.sz = np.array([[1, 0], [0, -1]]) / 2
        self.sx = np.array([[0, 1], [1, 0]]) / 2
        self.H = None

    def build_hamiltonian(self):
        from numpy import kron
        Iz_ops = []
        Ix_ops = []
        for i in range(self.n_spins):
            op_z = 1
            op_x = 1
            for j in range(self.n_spins):
                op_z = kron(op_z, self.sz if i == j else self.I)
                op_x = kron(op_x, self.sx if i == j else self.I)
            Iz_ops.append(op_z)
            Ix_ops.append(op_x)
        H = 0
        for i in range(self.n_spins):
            for j in range(i + 1, self.n_spins):
                J = self.j_couplings[0] if self.j_couplings else 7
                H += 2 * np.pi * J * Iz_ops[i] @ Iz_ops[j]
        for i in range(self.n_spins):
            H += 2 * np.pi * self.bx * Ix_ops[i]
        self.H = H
        return H

    def solve(self):
        if self.H is None:
            self.build_hamiltonian()
        eigvals, eigvecs = np.linalg.eigh(self.H)
        return eigvals, eigvecs

    # You can add methods for time evolution and animation as needed

---

## Summary

- **Reusable functions**: For Nyquist frequency, data loading, FFT, peak detection, integration, functional group identification, and multiplet analysis.
- **Classes**: `NMRSpectrum` for spectrum analysis and `SpinHamiltonian` for quantum simulation.
- **Next steps**: Move these functions and classes into a `.py` file (e.g., `nmr_utils.py`) for import into future notebooks or scripts.